In [1]:
import myfm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from myfm.utils.benchmark_data import MovieLens100kDataManager
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
import scipy.sparse as sps
from itertools import combinations
import numpy as np

In [2]:
movie_data_path = 'E:\\recommendation-system\\CONTROL\\movies_metadata.csv'
ratings_path = 'E:\\recommendation-system\\CONTROL\\ratings_small.csv'

In [3]:
df_movie = pd.read_csv(movie_data_path)
df_movie = df_movie.dropna(subset=['vote_count'])
df_movie.drop(columns=["title", "original_title", "poster_path","imdb_id","spoken_languages","homepage","tagline","overview"], axis=1,inplace=True)
df_movie['movie_id']= df_movie['movie_id'].astype('int')
df_ratings = pd.read_csv(ratings_path).drop(columns=['timestamp']).astype('object')
df_ratings['movie_id'] = df_ratings['movie_id'].astype('int')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11724\35809915.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movie = pd.read_csv(movie_data_path)


In [4]:
columns_movies = []
for col in df_movie.columns.values:
    if str(col) != 'movie_id':
        columns_movies.append('movie_'+str(col))
    else:
        columns_movies.append(str(col))
df_movie = pd.DataFrame(
    df_movie.values, columns=columns_movies)

In [10]:
df_movie

,movie_adult,movie_belongs_to_collection,movie_budget,movie_genres,movie_id,movie_original_language,movie_popularity,movie_production_companies,movie_production_countries,movie_release_date,movie_revenue,movie_runtime,movie_status,movie_video,movie_vote_average,movie_vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,Released,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,Released,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,Released,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,Released,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,Released,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45455,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,fa,0.072051,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,Released,False,4.0,1.0
45456,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,Released,False,9.0,3.0
45457,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,Released,False,3.8,6.0
45458,False,NaN,0,[],227506,en,0.003503,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,Released,False,0.0,0.0


In [5]:
data = pd.merge(df_ratings,df_movie,on='movie_id',how='outer',)
data['movie_id'] = data['movie_id'].astype('int')
data

c:\Python310\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


,user_id,movie_id,rating,movie_adult,movie_belongs_to_collection,movie_budget,movie_genres,movie_original_language,movie_popularity,movie_production_companies,movie_production_countries,movie_release_date,movie_revenue,movie_runtime,movie_status,movie_video,movie_vote_average,movie_vote_count
0,1,31,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7,31,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31,31,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32,31,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36,31,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142633,NaN,439050,NaN,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",fa,0.072051,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,Released,False,4.0,1.0
142634,NaN,111109,NaN,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",tl,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,Released,False,9.0,3.0
142635,NaN,67758,NaN,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",en,0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,Released,False,3.8,6.0
142636,NaN,227506,NaN,False,NaN,0,[],en,0.003503,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,Released,False,0.0,0.0


In [6]:
index_to_delete = data[data['user_id'].isna()].index
data.drop(index_to_delete, inplace=True)
data

,user_id,movie_id,rating,movie_adult,movie_belongs_to_collection,movie_budget,movie_genres,movie_original_language,movie_popularity,movie_production_companies,movie_production_countries,movie_release_date,movie_revenue,movie_runtime,movie_status,movie_video,movie_vote_average,movie_vote_count
0,1,31,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7,31,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31,31,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32,31,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36,31,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100004,664,64997,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,664,72380,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100006,665,129,3.0,False,NaN,15000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na...",ja,41.048867,"[{'name': 'Studio Ghibli', 'id': 10342}]","[{'iso_3166_1': 'JP', 'name': 'Japan'}]",2001-07-20,274925095.0,125.0,Released,False,8.3,3968.0
100007,665,4736,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data = data[~data['movie_vote_count'].isna()].reset_index().drop(columns=['index'])
data

,user_id,movie_id,rating,movie_adult,movie_belongs_to_collection,movie_budget,movie_genres,movie_original_language,movie_popularity,movie_production_companies,movie_production_countries,movie_release_date,movie_revenue,movie_runtime,movie_status,movie_video,movie_vote_average,movie_vote_count
0,1,1371,2.5,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,"[{'id': 18, 'name': 'Drama'}]",en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
1,4,1371,4.0,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,"[{'id': 18, 'name': 'Drama'}]",en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
2,7,1371,3.0,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,"[{'id': 18, 'name': 'Drama'}]",en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
3,19,1371,4.0,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,"[{'id': 18, 'name': 'Drama'}]",en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
4,21,1371,3.0,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,"[{'id': 18, 'name': 'Drama'}]",en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44989,652,129009,4.0,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",en,0.875177,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1963-04-24,0.0,111.0,Released,False,4.0,1.0
44990,653,2103,3.0,False,NaN,47000000,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",en,10.642824,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2002-11-27,30002758.0,99.0,Released,False,5.9,378.0
44991,659,167,4.0,False,NaN,48000000,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",en,12.400152,"[{'name': 'Intermedia Films', 'id': 763}, {'na...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",2001-10-22,50315140.0,120.0,Released,False,7.1,710.0
44992,659,563,3.0,False,"{'id': 10522, 'name': 'Starship Troopers Colle...",105000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",en,14.412644,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1997-11-06,121214377.0,129.0,Released,False,6.7,1584.0


In [12]:
# Xuất file csv để test import
df = pd.DataFrame(data)
df.to_csv('meta_data_merged.csv', sep=',', encoding='utf-8',index=False)

In [8]:
def change_to_true_dtype(data:pd.DataFrame):
    for i in data.columns:
        try:
            data[i] = pd.to_numeric(data[i])
        except ValueError:
            data[i] = data[i].astype('object')
    return data

In [42]:
test_data = data.copy()
# test_data = change_to_true_dtype(test_data)
test_data.dtypes

user_id                        object
movie_id                        int32
rating                         object
movie_adult                    object
movie_belongs_to_collection    object
movie_budget                   object
movie_genres                   object
movie_original_language        object
movie_popularity               object
movie_production_companies     object
movie_production_countries     object
movie_release_date             object
movie_revenue                  object
movie_runtime                  object
movie_status                   object
movie_video                    object
movie_vote_average             object
movie_vote_count               object
dtype: object

In [10]:
pd.api.types.is_numeric_dtype(test_data['movie_runtime'].dtypes)

True

In [79]:
a = eval(str(test_data['movie_production_companies'][0]))

In [67]:
eval(test_data['movie_belongs_to_collection'][0])['name']

'Rocky Collection'

In [80]:
type(a)

list

In [14]:
def sort_array(array):
    new_array = [i['name'] for i in eval(array)]
    new_array.sort()
    return new_array

In [12]:
test_data['movie_genres'] = test_data['movie_genres'].apply(lambda x: sort_array(x))
test_data['movie_genres']  = test_data['movie_genres'] .apply(lambda x: '|'.join([i.replace(" ","") for i in x]))
test_data

,user_id,movie_id,rating,movie_adult,movie_belongs_to_collection,movie_budget,movie_genres,movie_original_language,movie_popularity,movie_production_companies,movie_production_countries,movie_release_date,movie_revenue,movie_runtime,movie_status,movie_video,movie_vote_average,movie_vote_count
0,1,1371.0,2.5,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,Drama,en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
1,4,1371.0,4.0,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,Drama,en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
2,7,1371.0,3.0,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,Drama,en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
3,19,1371.0,4.0,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,Drama,en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
4,21,1371.0,3.0,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,Drama,en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44989,652,129009.0,4.0,False,NaN,0,Comedy|Romance,en,0.875177,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1963-04-24,0.0,111.0,Released,False,4.0,1.0
44990,653,2103.0,3.0,False,NaN,47000000,Drama|Mystery|Romance|ScienceFiction,en,10.642824,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2002-11-27,30002758.0,99.0,Released,False,5.9,378.0
44991,659,167.0,4.0,False,NaN,48000000,Drama|ScienceFiction,en,12.400152,"[{'name': 'Intermedia Films', 'id': 763}, {'na...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",2001-10-22,50315140.0,120.0,Released,False,7.1,710.0
44992,659,563.0,3.0,False,"{'id': 10522, 'name': 'Starship Troopers Colle...",105000000,Action|Adventure|ScienceFiction|Thriller,en,14.412644,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1997-11-06,121214377.0,129.0,Released,False,6.7,1584.0


In [13]:
test_data['movie_belongs_to_collection'] = test_data['movie_belongs_to_collection'].apply(lambda x: 'NaN' if (pd.isna(x)) else eval(str(x))['name'])
# test_data['movie_belongs_to_collection']  = test_data['movie_belongs_to_collection'] .apply(lambda x: ' '.join([i.replace(" ","") for i in x]))
test_data

,user_id,movie_id,rating,movie_adult,movie_belongs_to_collection,movie_budget,movie_genres,movie_original_language,movie_popularity,movie_production_companies,movie_production_countries,movie_release_date,movie_revenue,movie_runtime,movie_status,movie_video,movie_vote_average,movie_vote_count
0,1,1371.0,2.5,False,Rocky Collection,17000000,Drama,en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
1,4,1371.0,4.0,False,Rocky Collection,17000000,Drama,en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
2,7,1371.0,3.0,False,Rocky Collection,17000000,Drama,en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
3,19,1371.0,4.0,False,Rocky Collection,17000000,Drama,en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
4,21,1371.0,3.0,False,Rocky Collection,17000000,Drama,en,11.673366,"[{'name': 'United Artists', 'id': 60}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1982-05-28,270000000.0,99.0,Released,False,6.6,894.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44989,652,129009.0,4.0,False,NaN,0,Comedy|Romance,en,0.875177,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1963-04-24,0.0,111.0,Released,False,4.0,1.0
44990,653,2103.0,3.0,False,NaN,47000000,Drama|Mystery|Romance|ScienceFiction,en,10.642824,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2002-11-27,30002758.0,99.0,Released,False,5.9,378.0
44991,659,167.0,4.0,False,NaN,48000000,Drama|ScienceFiction,en,12.400152,"[{'name': 'Intermedia Films', 'id': 763}, {'na...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",2001-10-22,50315140.0,120.0,Released,False,7.1,710.0
44992,659,563.0,3.0,False,Starship Troopers Collection,105000000,Action|Adventure|ScienceFiction|Thriller,en,14.412644,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1997-11-06,121214377.0,129.0,Released,False,6.7,1584.0


In [63]:
test_data['movie_belongs_to_collection'].str.contains('{').values.shape[0]

44994

In [43]:
def change_to_true_dtype(data:pd.DataFrame):
    for i in data.columns:
        try:
            data[i] = pd.to_numeric(data[i])
        except ValueError:
            data[i] = data[i].astype('object')
    return data
    
def sort_array(array):
    new_array = [i['name'] for i in eval(array)]
    new_array.sort()
    return new_array
    
def check_value_is_dict(data:pd.DataFrame,column:str):
    for i in range(data[column].values.shape[0]):
        try:
            if type(eval(data[column][i])) == dict:
                return True
        except:
            return False
    return False
    
def check_value_is_list(data:pd.DataFrame,column:str):
    for i in range(data[column].values.shape[0]):
        try:
            if type(eval(data[column][i])) == list:
                return True
        except:
            return False
    return False
    
def prepare(data: pd.DataFrame):
    # if 'movie_release_date' in data_normalized.columns:
    #     data_normalized['movie_release_date'] = pd.to_datetime(
    #         data_normalized.movie_release_date)
    #     data_normalized['movie_release_year'] = [
    #         str(x) for x in data_normalized['movie_release_date'].dt.year.fillna('NaN')]
    # if "user_age" in data_normalized.columns:
    #     data_normalized["user_age"] = data_normalized.user_age.astype(int) // 5 * 5
    # if "user_zipcode" in data_normalized.columns:
    #     data_normalized["user_zipcode"] = data_normalized.user_zipcode.str[0]
    data = change_to_true_dtype(data)
    for i in data.columns:
        if 'id' in i and i not in ['movie_id', 'user_id']:
            data.drop(columns=[i], axis=1, inplace=True)
            continue
        if 'id' not in i:
            if 'date' in i:
                data[i] = pd.to_datetime(data[i])
                data[i] = [
                    str(x) for x in data[i].dt.year.fillna('NaN')]
                continue
            if pd.api.types.is_numeric_dtype(data[i].dtypes) and 'rating' not in i:
                data[i] = data[i] // 5 * 5
                continue
            if check_value_is_dict(data, i):
                data[i] = data[i].apply(lambda x: 'NaN' if (
                    pd.isna(x)) else eval(str(x))['name'])
                continue
            if check_value_is_list(data, i):
                data[i] = data[i].apply(lambda x: sort_array(x))
                data[i] = data[i].apply(lambda x: '|'.join([i.replace(" ","") for i in x]))
                continue
    return data.copy()


In [65]:
a = [1,2, 3, 4 ,5]
print(*a)

1 2 3 4 5


In [42]:
test_data['movie_genres'].str.contains('-',regex=False)

0        False
1        False
2        False
3        False
4        False
         ...  
44989    False
44990    False
44991    False
44992    False
44993    False
Name: movie_genres, Length: 44994, dtype: bool

In [11]:
def check_multi_label(data:pd.DataFrame,column:str):
    try:
        result = data[column].str.contains('|',regex=False)
        # print(result.index)
        for i in result.index:
            # print(result[i])
            if result[i] == True:
                return True
    except:
        return False
    return False

In [51]:
def test(x):
    print(x)
    return x

In [21]:
def multi_mle_to_multi_array(mle_list: list, column_list: list, df: pd.DataFrame):
    result = []
    for i in range(len(mle_list)):
        current_feature = column_list[i]
        current_column = current_feature.columns[0]
        array = mle_list[i].transform(current_feature[current_column].reindex(
            df['movie_id']).apply(lambda x: x.split('|')))
        result.append(array)
    return result


In [40]:
def train(data: pd.DataFrame):
    data = prepare(data)
    FEATURE_COLUMNS = ['user_id', 'movie_id']
    ohe = OneHotEncoder(handle_unknown='ignore')
    df_train = data
    # df_train.
    X_train = ohe.fit_transform(
        df_train[FEATURE_COLUMNS])
    y_train = df_train.rating.values
    # user features
    user_feature_name = [
        col for col in data.columns if 'user' in col]
    user_data = data[user_feature_name].drop_duplicates(
        subset="user_id").set_index('user_id')
    user_multi_label_list = []
    user_multi_label_mle_list = []
    for i in user_data.columns.values:
        if check_multi_label(user_data, i):
            column = user_data.loc[:, [i]]
            column_mle = MultiLabelBinarizer(sparse_output=True).fit(
                column[i].apply(lambda x: x.split('|'))
            )
            user_multi_label_list.append(column)
            user_multi_label_mle_list.append(column_mle)
            user_data.drop(columns=[i], axis=1, inplace=True)
    current_user_Data_ohe = OneHotEncoder(
        handle_unknown='ignore').fit(user_data)

    # movie features
    movie_feature_name = [
        col for col in data.columns if 'movie' in col]
    movie_data = data[movie_feature_name].drop_duplicates(
        subset="movie_id").set_index('movie_id')
    movie_multi_label_list = []
    movie_multi_label_mle_list = []
    for i in movie_data.columns.values:
        if check_multi_label(movie_data, i):
            column = movie_data.loc[:, [i]]
            column_mle = MultiLabelBinarizer(sparse_output=True).fit(
                column[i].apply(lambda x: x.split('|'))
            )
            movie_multi_label_list.append(column)
            movie_multi_label_mle_list.append(column_mle)
            movie_data.drop(columns=[i], axis=1, inplace=True)
    current_movie_Data_ohe = OneHotEncoder(
        handle_unknown='ignore').fit(movie_data)

    print(user_multi_label_list)
    print(user_multi_label_mle_list)
    # print(movie_multi_label_list[0])
    print(movie_multi_label_mle_list)
    # print(multi_mle_to_multi_array(movie_multi_label_mle_list,movie_multi_label_list,df_train))
    user_multi_label_mle_array = multi_mle_to_multi_array(user_multi_label_mle_list,user_multi_label_list,df_train)
    movie_multi_label_mle_array = multi_mle_to_multi_array(movie_multi_label_mle_list,movie_multi_label_list,df_train)
    X_train_extended = sps.hstack([
        X_train,
        current_user_Data_ohe.transform(
            user_data.reindex(df_train.user_id)
        ),
        current_movie_Data_ohe.transform(
            movie_data.reindex(
                df_train.movie_id)
        ),
        *user_multi_label_mle_array,
        *movie_multi_label_mle_array
        # movie_genre_mle.transform(
        #     movie_genre.movie_genres.reindex(
        #         df_train.movie_id).apply(lambda x: x.split('|'))
        # )
    ])
    group_shapes_extended = (
        [len(group) for group in ohe.categories_] +
        [len(group) for group in current_user_Data_ohe.categories_] +
        [len(group) for group in current_movie_Data_ohe.categories_]
        # [len(movie_genre_mle.classes_)]
    )
    for i in range(len(user_multi_label_mle_list)):
        current = user_multi_label_mle_list[i]
        group_shapes_extended = group_shapes_extended + [len(current.classes_)]

    for i in range(len(movie_multi_label_mle_list)):
        current = movie_multi_label_mle_list[i]
        group_shapes_extended = group_shapes_extended + [len(current.classes_)]

    print(group_shapes_extended)
    fm_side_info = myfm.MyFMRegressor(
            rank=10, random_seed=42,
        )
    fm_side_info.fit(
        X_train_extended, y_train, n_iter=150, n_kept_samples=150,
        group_shapes=group_shapes_extended
    )


In [18]:
# prepared_data = prepare(test_data)

In [44]:
train(test_data.copy())

[]
[]
[MultiLabelBinarizer(sparse_output=True), MultiLabelBinarizer(sparse_output=True), MultiLabelBinarizer(sparse_output=True)]
[671, 2830, 2, 346, 311, 38, 15, 105, 1166, 52, 5, 3, 442, 21, 3181, 86]


alpha = 1.91 w0 = 3.94 : 100%|██████████| 150/150 [00:29<00:00,  5.16it/s]


In [2]:
data_manager=MovieLens100kDataManager()

In [3]:
# print(dataset.load_rating_predefined_split(3))
ratings = data_manager.load_rating_all().drop(columns=['timestamp'])
users = data_manager.load_user_info()
columns_users = []
for col in users.columns:
    if str(col) != 'user_id' :
        columns_users.append('user_'+str(col))
    else:
        columns_users.append(str(col))
users = pd.DataFrame(users.values,columns=columns_users)
#Merge tập ratings và users được gom nhóm bởi user_id
result =  pd.merge(ratings, users, on="user_id", how="outer")

movies = data_manager.load_movie_info()
columns_movies = []
for col in movies[['movie_id','release_date','genres']].columns:
    if str(col) != 'movie_id' :
        columns_movies.append('movie_'+str(col))
    else:
        columns_movies.append(str(col))
movies = pd.DataFrame(movies[['movie_id','release_date','genres']].values,columns=columns_movies)
result=  pd.merge(result, movies, on="movie_id", how="outer")

In [4]:
result

,user_id,movie_id,rating,user_age,user_gender,user_occupation,user_zipcode,movie_release_date,movie_genres
0,196,242,3,49,M,writer,55105,1997-01-24,Comedy
1,305,242,5,23,M,programmer,94086,1997-01-24,Comedy
2,6,242,4,42,M,executive,98101,1997-01-24,Comedy
3,234,242,4,60,M,retired,94702,1997-01-24,Comedy
4,63,242,3,31,M,marketing,75240,1997-01-24,Comedy
...,...,...,...,...,...,...,...,...,...
99995,863,1679,3,17,M,student,60089,1998-02-06,Romance|Thriller
99996,863,1678,1,17,M,student,60089,1998-02-06,Drama
99997,863,1680,2,17,M,student,60089,1998-01-01,Drama|Romance
99998,896,1681,3,28,M,writer,91505,1994-01-01,Comedy


In [10]:
def get_user_info(result):
    user_feature_name = [col for col in result.columns if 'user' in col]
    user_data = result[user_feature_name].drop_duplicates(subset = "user_id").set_index('user_id').sort_values(by='user_id')
    return user_data

In [11]:
get_user_info(result)

c:\Python310\lib\site-packages\pandas\core\indexes\base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,user_age,user_gender,user_occupation,user_zipcode
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213
...,...,...,...,...
939,26,F,student,33319
940,32,M,administrator,02215
941,20,M,student,97229


In [12]:
def get_movie_info(result):
    movie_feature_name = [col for col in result.columns if 'movie' in col]
    movie_data = result[movie_feature_name].drop_duplicates(subset = "movie_id").set_index('movie_id').sort_values(by='movie_id')
    return movie_data

In [13]:
get_movie_info(result)

c:\Python310\lib\site-packages\pandas\core\indexes\base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,movie_release_date,movie_genres
movie_id,,
1,1995-01-01,Animation|Children's|Comedy
2,1995-01-01,Action|Adventure|Thriller
3,1995-01-01,Thriller
4,1995-01-01,Action|Comedy|Drama
5,1995-01-01,Crime|Drama|Thriller
...,...,...
1678,1998-02-06,Drama
1679,1998-02-06,Romance|Thriller
1680,1998-01-01,Drama|Romance
